In [1]:
%pip install ultralytics
%pip install super-gradients
%pip install supervision
%pip install torch

In [2]:
from ultralytics import NAS

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from super_gradients.training import models
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

best_model = models.get(
    'yolo_nas_l',
    num_classes=1,
    checkpoint_path="/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/average_model.pth"
).to(DEVICE)


The console stream is logged into /root/sg_logs/console.log


[2023-12-20 13:08:09] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-12-20 13:08:09] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-12-20 13:08:18] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-12-20 13:08:18] WARNING - export.py - Failed to import pytorch_quantization
[2023-12-20 13:08:18] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-12-20 13:08:24] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/average_model.pth EMA checkpoint.


In [5]:
import supervision as sv

def get_inference(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )

  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = list(best_model.predict(image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections
    nome_imagem = image_name.split('/')[7] + ".txt"
    with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/inferencias/{nome_imagem}","w") as file:
      file.write(str(detections.xyxy))

get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2023-12-20 13:04:05] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:04:07] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:04:07] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:04:08] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:04:08] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:04:08] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:04:09] INFO -

In [6]:
import supervision as sv
box_annotator = sv.BoxAnnotator()

images = []
titles = []

detections_list = []
prediction_list = []

def get_ioudata(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )
  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = list(best_model.predict(image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

  keys = list(ds.images.keys())
  for key in keys:
      # Para anotações
      annotation_detections = ds.annotations[key]
      for detection in annotation_detections:
          detections_list.append(detection[0])

      # Para previsões
      prediction_detections = predictions[key]
      for prediction in prediction_detections:
          prediction_list.append(prediction[0])

get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2023-12-20 13:09:28] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:09:31] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:09:31] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:09:31] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:09:32] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:09:32] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2023-12-20 13:09:32] INFO -

In [7]:
import numpy as np


def iou(box1, box2):
    # Verifica se as bounding boxes são válidas.
    if box1[2] is None or box2[2] is None:
        return 0.0

    # Calcula as interseções das bounding boxes.
    inter_top = np.maximum(box1[0], box2[0])
    inter_bottom = np.minimum(box1[2], box2[2])
    inter_left = np.maximum(box1[1], box2[1])
    inter_right = np.minimum(box1[3], box2[3])
    inter_area = (inter_right - inter_left) * (inter_bottom - inter_top)

    # Calcula as áreas das bounding boxes.
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    # Calcula o IoU.
    iou = inter_area / (box1_area + box2_area - inter_area)

    return iou

for i in range(len(detections_list)):
  iou_score = iou(detections_list[i], prediction_list[i])
  with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/Iou.txt",'a') as file:
    file.write(str(iou_score)+ '\n')
  print(iou_score)

0.8845069
0.93238854
0.8266967
0.87802154
0.7916934
0.8446695
0.96136683
0.8727609
0.83443916
0.8763265
0.88084704
0.8324876
0.85014606
0.8654502
-0.105946355
-0.0140053835
0.7298849
0.4547523
-0.039601024
0.011051011
-0.338128
-0.0075828377
0.04764998
0.18003891
-0.025478967
0.01872536
0.817662
-0.12458334
-0.3140405
0.14896415
0.47825864
0.23197694
-0.0062451963
-0.014813169
-0.08834073
0.040748842
-0.38164327
0.039290044
0.027459593
0.017116174
-0.075061426
0.14480403
0.4608537
0.020064386
0.056296665
-0.2312827
-0.019054767
0.65424275
-0.002201373
0.17200737
-0.013104049
-0.0008545565
-0.011791547
0.15933196
0.15307891
0.5327867
-0.052345008
0.96548617
-0.07551323
-0.014367754
-1.5742995
0.017061915
0.9422589
0.2694897
0.52286863
0.033090062
-0.122730024
0.008852741
0.31806013
-0.0533883
-0.03555705
-0.036291424
0.11745538
0.077702805
-0.10452128
-0.027906535
-0.08133475
0.20164207
0.23207302
-0.09715601
0.014891793
0.05800958
0.013156691
0.8902188
-0.06936031
-0.040858883
-0.31494

In [8]:
detections_list[1]

array([     294.62,      118.46,      383.08,       227.7], dtype=float32)